# Helpful links
- https://stackoverflow.com/questions/15007304/histogram-equalization-not-working-on-color-image-opencv
- https://docs.opencv.org/master/d7/d1b/group__imgproc__misc.html#ga72b913f352e4a1b1b397736707afcde3

In [ ]:
# mandatory imports
import cv2 as cv
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt

from glob import glob

import sys
sys.path.append("..") # Adds higher directory to python modules path.

from utils.dataset_2_utils import find_laser_beams

In [ ]:
DATA_PATH = r'/scratch/gallowaa/cciw/'  # MLRG workstation
#DATA_PATH = r'/media/angus/cciw/'  # Laptop

In [ ]:
# Search for all image files in testing set...
all_images = glob(
    osp.join(DATA_PATH, 'Data/Videos_and_stills/TestingSet/WHERD/*/*/*/*/*.jpg'))
print(len(all_images))

In [ ]:
#save_path = osp.join('/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x/Test/WHERD')
save_path = osp.join(DATA_PATH, 'dataset_raw/Test/WHERD-unlabeled')
print(save_path)

In [ ]:
"""Note: Names differ slightly from email with subject line 
'Selection of images to label for testing set' sent by 
dominique.brunet@canada.ca Wed 2/12/2020 4:04 PM, 
add 'FS' prefix to images taken with FishSense camera.
"""

# files to include
include_list = [
    'WHERD_1353_2019-10-30_GoPro-8.jpg',
    'WHERD_1355_2019-10-30_GoPro-5.jpg',
    'WHERD_FS1355_2019-11-03_FishSens-1.jpg',  # (100% mussel coverage)
    'WHERD_HP115410_2019-10-30_GoPro-3.jpg',
    'WHERD_HP131641_2019-11-03_GoPro-13.jpg',
    'WHERD_FSHP131642_2019-11-03_FishSens-4.jpg',
    'WHERD_FSHP131645_2019-11-03_FishSens-6.jpg',
    'WHERD_FSHP131649_2019-11-03_FishSens-2.jpg',
    'WHERD_LEE06021_2019-10-30_GoPro-3.jpg',
    'WHERD_FSLEE06022_2019-11-03_FishSens-1.jpg',
    'WHERD_LEE06031_2019-10-30_GoPro-2.jpg',
    'WHERD_LEE06032_2019-11-03_GoPro-3.jpg',
    'WHERD_LEE06033_2019-10-30_GoPro-8.jpg',
    'WHERD_FSLEE06037_2019-11-03_FishSens-1.jpg',
    'WHERD_LEE06070_2019-10-30_GoPro-1.jpg',  # (no mussels)
    'WHERD_LEE06075_2019-10-30_GoPro-11.jpg'
]
print(len(include_list))

In [ ]:
j = 0
for i in range(len(all_images)):
    f = all_images[i]
    if f.split('/')[-1] in include_list:
        #outfile = os.path.join(save_path, f.split('/')[-1])
        
        im = cv.imread(f)  # im will be in BGR format
        coords = find_laser_beams(im)
        d = np.linalg.norm(coords[0] - coords[1])
        
        # draw the two clusters
        if (coords != -1).any():
            for pt in coords:
                _ = cv.circle(im, (int(pt[0]), int(pt[1])), 50, (0, 255, 0), 2) 

        fig, ax = plt.subplots(1, 1, figsize=(14, 6))
        title_str = '%d Ratio %.3f - dots are dist %d of %d px' % (
            j, d/im.shape[1], d, im.shape[1])
        ax.imshow(im)
        ax.set_title(title_str)
        plt.show()
        j += 1